# Preferences Research dataset quiz

## Environment

### Libraries

In [1]:
! pip install SQLAlchemy==1.4.31 > /dev/null
! pip install pandas==1.4.1 > /dev/null
! pip install mysql-connector-python==8.0.28 > /dev/null

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine

### DB Connection

In [3]:
def get_engine_url_from_env():
    env_vars = ['DB_ADAPTER', 'DB_HOST', 'DB_PORT', 'DB_USERNAME', 'DB_PASSWORD', 'DB_NAME']
    adapter, host, port, username, password, database = [os.environ.get(env_var) for env_var in env_vars]
    return f'{adapter}://{username}:{password}@{host}:{port}/{database}'

In [4]:
engine = create_engine(get_engine_url_from_env())
conn = engine.connect()

## Quiz

### 1. How many womans likes cat?

In [5]:
pd.read_sql("""
        SELECT COUNT(DISTINCT person_id) AS total
          FROM research
  NATURAL JOIN person
  NATURAL JOIN pet
         WHERE person_gender = 'Feminino'
           AND pet_animal = 'gato'
""", con=conn)

,total
0,279


### 2. Which is the mean age of men who likes tea and cold weather?

In [6]:
pd.read_sql("""
        SELECT AVG(TIMESTAMPDIFF(YEAR, person_birthdate, CURDATE())) AS mean
          FROM research
  NATURAL JOIN person
  NATURAL JOIN drink
  NATURAL JOIN weather
         WHERE person_gender = 'Masculino'
           AND drink_description = 'Chá'
           AND weather_description = 'frio'
""", con=conn)

,mean
0,43.9359


### 3. Which is the preferred men hobby?

In [8]:
pd.read_sql("""
        SELECT hobby_description AS hobby,
               COUNT(*) AS total
          FROM research
  NATURAL JOIN person
  NATURAL JOIN hobby
         WHERE person_gender = 'Masculino'
      GROUP BY hobby
      ORDER BY total DESC
         LIMIT 1
""", con=conn)

,hobby,total
0,Dormir,184


### 4. Which is the preferred hobby of women who likes turtle?

In [9]:
pd.read_sql("""
        SELECT hobby_description AS hobby,
               COUNT(*) AS total
          FROM research
  NATURAL JOIN person
  NATURAL JOIN hobby
  NATURAL JOIN pet
         WHERE person_gender = 'Feminino'
           AND pet_animal = 'tartaruga'
      GROUP BY hobby
      ORDER BY total DESC
         LIMIT 1
""", con=conn)

,hobby,total
0,Escrever,25


### 5. Which is the favorite drink of women and men, respectivelly?

In [10]:
pd.read_sql("""
        SELECT * FROM (
            SELECT person_gender as gender,
                   drink_description as drink,
                   COUNT(*) AS total
              FROM research
      NATURAL JOIN person
      NATURAL JOIN drink
             WHERE person_gender = 'Feminino'
          GROUP BY drink
          ORDER BY total DESC
             LIMIT 1) AS women
               
         UNION
         
        SELECT * FROM (
            SELECT person_gender as gender,
                   drink_description as drink,
                   COUNT(*) AS total
              FROM research
      NATURAL JOIN person
      NATURAL JOIN drink
             WHERE person_gender = 'Masculino'
          GROUP BY drink
          ORDER BY total DESC
             LIMIT 1) AS men
""", con=conn)

,gender,drink,total
0,Feminino,Cerveja,138
1,Masculino,Café,258


### 6. Which is the mean age of people whose hobby is write?

In [11]:
pd.read_sql("""
        SELECT AVG(TIMESTAMPDIFF(YEAR, person_birthdate, CURDATE())) AS mean
          FROM research
  NATURAL JOIN person
  NATURAL JOIN hobby
         WHERE hobby_description = 'Escrever'
""", con=conn)

,mean
0,47.8333


### 7. How many people watches TV as hobby and which is your mean age, respectively?

In [12]:
pd.read_sql("""
        SELECT COUNT(*) AS total,
               AVG(TIMESTAMPDIFF(YEAR, person_birthdate, CURDATE())) AS mean
          FROM research
  NATURAL JOIN person
  NATURAL JOIN hobby
         WHERE hobby_description = 'Assistir TV'
""", con=conn)

,total,mean
0,263,45.6882


## Finishing DB Connection

In [ ]:
conn.close()